<a href="https://colab.research.google.com/github/Zfeng0207/FIT3199-FYP/blob/dev%2Fdarin/FYP_LSTM_DL_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
import pandas as pd
import numpy as np
import gdown

#Import Data from Google Drive

In [17]:
file_id = "1CsLg1kROZWRtGXBJ1YHK_CW4XbBqSI3P"
url = f"https://drive.google.com/uc?id={file_id}"
gdown.download(url, "health_data.csv", quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1CsLg1kROZWRtGXBJ1YHK_CW4XbBqSI3P
To: /content/health_data.csv
100%|██████████| 5.92M/5.92M [00:00<00:00, 147MB/s]


'health_data.csv'

In [18]:
data = pd.read_csv('health_data.csv')
data

,subject_id,stay_id_x,charttime,temperature,heartrate,resprate,o2sat,sbp,dbp,rhythm,...,seq_num,icd_code,icd_version,icd_title,Stroke_Y/N,gender,anchor_age,anchor_year,anchor_year_group,dod
0,14394983,34259230.0,2202-07-10 12:24:00,98.3,74.0,19.0,97.0,153.0,67.0,-1,...,2,262,10,369,0,1,46,2196,0,NaN
1,17730806,30258441.0,2182-07-17 16:58:00,NaN,82.0,16.0,96.0,115.0,102.0,-1,...,1,58,9,256,1,1,68,2176,1,NaN
2,15973805,30481231.0,2145-07-03 17:20:00,98.0,109.0,18.0,98.0,113.0,77.0,-1,...,2,471,9,452,0,0,51,2139,1,NaN
3,16945691,36369419.0,2117-12-18 23:50:00,99.9,74.0,16.0,100.0,151.0,76.0,-1,...,1,388,10,158,0,0,81,2111,1,2118-09-20
4,15632719,33618002.0,2170-04-19 20:58:00,98.4,83.0,NaN,94.0,142.0,81.0,-1,...,1,58,9,256,1,1,55,2170,2,2170-10-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52285,15853169,33648217.0,2174-04-13 17:56:00,98.1,62.0,16.0,100.0,121.0,62.0,-1,...,2,330,10,482,1,0,52,2170,2,NaN
52286,10913302,37712176.0,2196-11-26 09:23:00,NaN,NaN,NaN,NaN,NaN,NaN,-1,...,1,403,10,203,0,1,25,2185,0,NaN
52287,11553072,31817224.0,2164-12-23 16:45:00,99.8,92.0,16.0,96.0,160.0,80.0,-1,...,1,34,9,34,0,1,46,2156,0,NaN
52288,10295929,35519857.0,2150-12-05 06:15:00,97.9,63.0,20.0,99.0,149.0,97.0,-1,...,3,462,9,219,1,1,61,2141,0,NaN


In [19]:
# Drop non-relevant columns
columns_to_drop = ["subject_id", "stay_id_x", "stay_id_y", "charttime", "dod", "icd_title"]
data = data.drop(columns=columns_to_drop, errors='ignore')

# Handle missing values by filling with column mean
data = data.fillna(data.mean())

In [20]:
# Our Stroke Target Column
target_column = "Stroke_Y/N"

# Extract features and target
X = data.drop(columns=[target_column]).values
y = data[target_column].values

In [21]:
# Convert to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32).unsqueeze(1)  # Shape: (N,1)

In [22]:
# Split into train, validation, and test sets
train_size = int(0.7 * len(X_tensor))
val_size = int(0.15 * len(X_tensor))
test_size = len(X_tensor) - train_size - val_size
train_data, val_data, test_data = random_split(TensorDataset(X_tensor, y_tensor), [train_size, val_size, test_size])

In [23]:
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

#LSTM Model

In [24]:
# Define an LSTM Model for Stroke Prediction
class StrokeLSTM(nn.Module):
    def __init__(self, input_size, hidden_size=64, num_layers=2):
        super(StrokeLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = x.unsqueeze(1)  # Reshape for LSTM (batch, seq, feature)
        lstm_out, _ = self.lstm(x)
        x = self.fc(lstm_out[:, -1, :])  # Use last LSTM output
        return self.sigmoid(x)

In [27]:
# Model parameters
input_size = X.shape[1]  # Number of features
model = StrokeLSTM(input_size)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


#Testing Model

In [25]:
# Training loop
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=20):
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            y_pred = model(X_batch)
            loss = criterion(y_pred, y_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for X_val, y_val in val_loader:
                y_val_pred = model(X_val)
                val_loss += criterion(y_val_pred, y_val).item()

        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {total_loss/len(train_loader):.4f}, Val Loss: {val_loss/len(val_loader):.4f}")

In [26]:
train_model(model, train_loader, val_loader, criterion, optimizer, epochs=20)

Epoch 1/20, Train Loss: 0.6287, Val Loss: 0.5730
Epoch 2/20, Train Loss: 0.4836, Val Loss: 0.4421
Epoch 3/20, Train Loss: 0.4490, Val Loss: 0.4148
Epoch 4/20, Train Loss: 0.4367, Val Loss: 0.4434
Epoch 5/20, Train Loss: 0.4360, Val Loss: 0.4083
Epoch 6/20, Train Loss: 0.4288, Val Loss: 0.4298
Epoch 7/20, Train Loss: 0.4289, Val Loss: 0.5189
Epoch 8/20, Train Loss: 0.4418, Val Loss: 0.4346
Epoch 9/20, Train Loss: 0.4204, Val Loss: 0.4460
Epoch 10/20, Train Loss: 0.4270, Val Loss: 0.4109
Epoch 11/20, Train Loss: 0.4237, Val Loss: 0.4242
Epoch 12/20, Train Loss: 0.4201, Val Loss: 0.4125
Epoch 13/20, Train Loss: 0.4219, Val Loss: 0.4815
Epoch 14/20, Train Loss: 0.4243, Val Loss: 0.4168
Epoch 15/20, Train Loss: 0.4224, Val Loss: 0.4136
Epoch 16/20, Train Loss: 0.4327, Val Loss: 0.4038
Epoch 17/20, Train Loss: 0.4234, Val Loss: 0.4128
Epoch 18/20, Train Loss: 0.4196, Val Loss: 0.4794
Epoch 19/20, Train Loss: 0.4161, Val Loss: 0.4209
Epoch 20/20, Train Loss: 0.4145, Val Loss: 0.4213


#Evaluate Model

In [28]:
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for X_test, y_test in test_loader:
            y_test_pred = model(X_test)
            predicted = (y_test_pred > 0.5).float()
            correct += (predicted == y_test).sum().item()
            total += y_test.size(0)
    print(f'Test Accuracy: {correct / total:.4f}')

evaluate_model(model, test_loader)


Test Accuracy: 0.4262
